In [1]:
import os

In [4]:
import re
import pandas as pd
from sf_utils_snowpark import CONN_PARAM
from snowflake.connector.pandas_tools import write_pandas

In [5]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, FloatType, StringType, DateType
from snowflake.snowpark.functions import avg, sum, col, udf, call_udf, call_builtin, year
from datetime import date, datetime

In [6]:
session = Session.builder.configs(CONN_PARAM).create()

You need to create a UDF that will produce a fiscal year for the start_date. The logic should be that if the month is from May onwards* then the fiscal year should be the current year + 1, otherwise, the current year.

For example:

2022-05-13 = FY23

2022-02-11=FY22

In [7]:
@udf(name="arbitrary_fy",
     is_permanent=True,
     stage_location="@DVD_FROSTY_FRIDAYS_29",
     return_type=StringType(),
     input_types=[DateType()],
     packages= ["pandas"],
     replace=True,
     session=session)
def arbitrary_fy(target_date):
    '''Expects a datetime.date object'''
#     return 'sample_done'
    return 'FY'+ (str(target_date.year) if target_date.month < 5 else str(target_date.year + 1)) 